# Agent Tracking with AgentOps

<img src="https://github.com/AgentOps-AI/agentops/blob/main/docs/images/external/logo/banner-badge.png?raw=true" width="25%"/>

[AgentOps](https://agentops.ai/?=autogen) provides session replays, metrics, and monitoring for AI agents.

At a high level, AgentOps gives you the ability to monitor LLM calls, costs, latency, agent failures, multi-agent interactions, tool usage, session-wide statistics, and more. For more info, check out the [AgentOps Repo](https://github.com/AgentOps-AI/agentops).


### Overview Dashboard
<img src="https://raw.githubusercontent.com/AgentOps-AI/agentops/main/docs/images/external/app_screenshots/overview.gif"  width="25%"/>

### Session Replays
<img src="https://raw.githubusercontent.com/AgentOps-AI/agentops/main/docs/images/external/app_screenshots/drilldown.gif"  width="25%"/>

## Adding AgentOps to an existing Autogen service.
To get started, you'll need to install the AgentOps package and set an API key.

AgentOps automatically configures itself when it's initialized meaning your agent run data will be tracked and logged to your AgentOps account right away.

First let's install the required packages

In [ ]:
%pip install -U pyautogen
%pip install -U agentops
%pip install -U python-dotenv

Then import them

In [ ]:
from autogen import ConversableAgent, UserProxyAgent
from typing import Annotated, Literal
from autogen import ConversableAgent, register_function
import agentops
import os
from os import environ
from dotenv import load_dotenv
from IPython.core.error import (
    StdinNotImplementedError,
)  # only needed by AgentOps testing automation

Next, we'll set our API keys. There are several ways to do this, the code below is just the most foolproof way for the purposes of this notebook. It accounts for both users who use environment variables and those who just want to set the API Key here in this notebook.

[Get an AgentOps API key](https://agentops.ai/settings/projects)

1. Create an environment variable in a .env file or other method. By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Or...

2. Replace `<your_agentops_key>` below and pass in the optional `api_key` parameter to the AgentOps `init(api_key=...)` function. Remember not to commit your API key to a public repo!

In [ ]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") or "<your_openai_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

### Set an API key

By default, the AgentOps `init()` function will look for an environment variable named `AGENTOPS_API_KEY`. Alternatively, you can pass one in as an optional parameter.

Create an account and obtain an API key at [AgentOps.ai](https://agentops.ai/settings/projects)

In [ ]:
# When initializing AgentOps, you can pass in optional tags to help filter sessions.
agentops.init(AGENTOPS_API_KEY, default_tags=["simple-autogen-example"])

Autogen will now start automatically tracking
- LLM prompts and completions
- Token usage and costs
- Agent names and actions
- Correspondence between agents
- Tool usage
- Errors

# Simple Chat Example

In [ ]:
# Define model, openai api key, tags, etc in the agent configuration.
config_list = [
    {"model": "gpt-4-turbo", "api_key": OPENAI_API_KEY, "tags": ["gpt-4", "tool"]}
]
# Create the agent that uses the LLM.
assistant = ConversableAgent("agent", llm_config={"config_list": config_list})

# Create the agent that represents the user in the conversation.
user_proxy = UserProxyAgent("user", code_execution_config=False)

# Let the assistant start the conversation.  It will end when the user types "exit".
try:
    assistant.initiate_chat(user_proxy, message="How can I help you today?")
except StdinNotImplementedError:
    # This is only necessary for AgentOps testing automation which is headless and will not have user input
    print("Stdin not implemented. Skipping initiate_chat")
    agentops.end_session("Indeterminate")

# Close your AgentOps session to indicate that it completed.
agentops.end_session("Success")

![session replay](https://github.com/AgentOps-AI/agentops/blob/main/docs/images/external/app_screenshots/session-overview.png?raw=true)

# Tool Example
AgentOps also tracks when Autogen agents use tools. You can find more information on this example in [tool-use.ipynb](https://github.com/microsoft/autogen/blob/main/website/docs/tutorial/tool-use.ipynb)

In [ ]:
config_list = [
    {"model": "gpt-4-turbo", "api_key": OPENAI_API_KEY, "tags": ["gpt-4", "tool"]}
]

Operator = Literal["+", "-", "*", "/"]


def calculator(a: int, b: int, operator: Annotated[Operator, "operator"]) -> int:
    if operator == "+":
        return a + b
    elif operator == "-":
        return a - b
    elif operator == "*":
        return a * b
    elif operator == "/":
        return int(a / b)
    else:
        raise ValueError("Invalid operator")


# Create the agent that uses the LLM.
assistant = ConversableAgent(
    name="Assistant",
    system_message="You are a helpful AI assistant. "
    "You can help with simple calculations. "
    "Return 'TERMINATE' when the task is done.",
    llm_config={"config_list": config_list, "cache_seed": None},
)

# The user proxy agent is used for interacting with the assistant agent
# and executes tool calls.
user_proxy = ConversableAgent(
    name="User",
    llm_config=False,
    is_termination_msg=lambda msg: msg.get("content") is not None
    and "TERMINATE" in msg["content"],
    human_input_mode="NEVER",
)

assistant.register_for_llm(name="calculator", description="A simple calculator")(
    calculator
)
user_proxy.register_for_execution(name="calculator")(calculator)

# Register the calculator function to the two agents.
register_function(
    calculator,
    caller=assistant,  # The assistant agent can suggest calls to the calculator.
    executor=user_proxy,  # The user proxy agent can execute the calculator calls.
    name="calculator",  # By default, the function name is used as the tool name.
    description="A simple calculator",  # A description of the tool.
)

# Let the assistant start the conversation.  It will end when the user types "exit".
user_proxy.initiate_chat(assistant, message="What is (1423 - 123) / 3 + (32 + 23) * 5?")

agentops.end_session("Success")

You can see your run in action at [app.agentops.ai](https://app.agentops.ai). In this example, the AgentOps dashboard will show:
- Agents talking to each other
- Each use of the `calculator` tool
- Each call to OpenAI for LLM use

![Session Drilldown](https://github.com/AgentOps-AI/agentops/blob/main/docs/images/external/app_screenshots/session-replay.png?raw=true)